# Solving the Incompressible Semi-Geostrophic Problem in 3D

First load in the required packages and paths.

In [ ]:
import initialconditions as ic
import matplotlib.pyplot as plt
import numpy as np
from solvers import mainAB2 
from solvers import mainHeun 
from solvers import mainRK4 
from solvers import mainDOPRI 
from solvers import mainAB2wWa 
from solvers import mainHeunAB2
from solvers import mainCN

import random

Next define the system parameters.

In [ ]:
# Define the parameters of the system

# box = [-1, -1, -1, 1, 1, 1]
box = [-3.66, -1.75, 0, 3.66, 1.75, 0.45] # List or tuple defining domain [xmin, ymin, zmin, xmax, ymax, zmax]
per_tol = 1e-3 # Percent tolerance
per_x = True # Set the periodicity of X
per_y = False # Set the periodicity of Y
per_z = False # Set the periodicity of Z
tf = 52 # Final time
Ndt = 1207 # Number of timesteps

Define the initial condition and visualise it.

In [ ]:
# Define the parameters and initialize an initial condition that is a perturbation of a steady background state

# N = 1000
# B = np.array([[1, 2, 3], [2, 4, 5], [3, 5, 6]]) #Create background steady state
# Z = ic.create_ss_initial(N, B, box, 'Thermal Sine') #Initial seed positions as a perturbation of a steady state

# Define the parameters and initialize an initial condition that is an isolated cyclone

N = 40 ** 3 # Number of seeds
A = -0.5 # Shear parameter can either be 0 or +/-0.1
Z = ic.create_cyc_initial(N, box, A, per_x, per_y, per_z, truncation = 16) #Initial seed positions for an isolated cyclone with no shear, shear can be set to +/-0.1

# N = 10
# Z = np.array([(random.uniform(box[0], box[3]),
#               random.uniform(box[1], box[4]),
#               random.uniform(box[2], box[5])) for _ in range(N)]) # Place the seeds randomly

# Plot the initial condition

fig = plt.figure()
fig.set_size_inches(10, 10, True)
ax = fig.add_subplot(projection='3d')
ax.scatter(Z[:,0], Z[:,1], Z[:,2], c = Z[:,2], cmap = 'jet', edgecolor = 'none', s = 8) #Points colored based on their 3rd component i.e. temperature
ax.set_xlim([np.min(Z[:,0]), np.max(Z[:,0])])
ax.set_ylim([np.min(Z[:,1]), np.max(Z[:,1])])
ax.set_zlim([np.min(Z[:,2]), np.max(Z[:,2])])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev = 90, azim = -90) #Viewing angle on the inital conditions

ax.set_zticks([])  
ax.zaxis.label.set_visible(False) 
ax.xaxis.pane.set_visible(False)  
ax.yaxis.pane.set_visible(False) 
ax.zaxis.pane.set_visible(False) 
ax.grid(False) 

plt.show()

Solve the problem and save the data

In [ ]:
# mainAB2.SG_solver(box, Z, per_tol, tf, Ndt, per_x, per_y, per_z, debug = True) #Solve and save the solution 
# mainHeun.SG_solver(box, Z, per_tol, tf, Ndt, per_x, per_y, per_z, debug = True) #Solve and save the solution 
# mainRK4.SG_solver(box, Z, per_tol, tf, Ndt, per_x, per_y, per_z, debug = True) #Solve and save the solution 
# mainDOPRI.SG_solver(box, Z, per_tol, tf, Ndt, per_x, per_y, per_z, debug = True) #Solve and save the solution 
# mainAB2wWa.SG_solver(box, Z, per_tol, tf, Ndt, per_x, per_y, per_z, debug = True) #Solve and save the solution 
mainCN.SG_solver(box, Z, per_tol, tf, Ndt, per_x, per_y, per_z, debug = True) #Solve and save the solution

#Can activate the optional variables solver, and debug to control which linear solver the code uses and whether or not the code is in debug mode.

# Animations

First animate the particles

In [ ]:
import animators as ani

# Animate the particles 

ani.point_animator('./data/RK4_SG_data_1207_19683.msgpack', 'C', '2D', box, tf) #Animate the seeds or centroids depending on choice of 'Z' or 'C' and '2D' or '3D'
#ani.point_animator('./data/SG_data.msgpack', 'C', '3D', box, tf) #Animate the seeds or centroids depending on choice of 'Z' or 'C' and '2D' or '3D'
ani.point_animator('./data/RK4_SG_data_1207_19683.msgpack', 'Z', '2D', box, tf) #Animate the seeds or centroids depending on choice of 'Z' or 'C' and '2D' or '3D'
#ani.point_animator('./data/SG_data.msgpack', 'Z', '3D', box, tf) #Animate the seeds or centroids depending on choice of 'Z' or 'C' and '2D' or '3D'

Next load in the data to animate the cells

In [ ]:
import auxfunctions as aux

# Load the data from the MessagePack file
Z, C, W, M, TC = aux.load_data('./PaperData/RK4_SG_data_A=-0.5.msgpack')

# Compute the Velocities, Temperature, and Kinetic Energy
MVel, ZVel, TVel, T, E, Eerr = aux.get_properties(Z, C, TC)

In [ ]:
# Calculate time in days for each timestep
time_steps = np.arange(len(E))
days = time_steps / 48.0  # Convert timesteps to days

# Create a line plot
plt.plot(days, E)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))

# Add labels and title
plt.xlabel('Time (Days)')
plt.ylabel('Fluctuations in Total Energy')
plt.ylim(0, 4)
plt.title('Evolution of Fluctuations in Total Energy')

# Save the figure
plt.savefig('total_energy_flux_evolution_3D.png', dpi=300)  # Save as high-resolution PNG file

plt.show()

Plot the Seeds and Centroids

In [ ]:
import animators as ani

bounds = ani.get_animation_bounds(Z)
fig = plt.figure()
fig.set_size_inches(10, 10, True)
ax = fig.add_subplot()
for i in [192, 384, 576, 768, 960, 1200]:
    scatter = ax.scatter(Z[i][:,0], Z[i][:,1], c=Z[i][:,2], cmap='jet', edgecolor='none', s=10) 
    ax.set_xlim([bounds[0], bounds[3]])
    ax.set_ylim([bounds[1], bounds[4]])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')

    # Calculate day number
    day_id = i // 48  # Using integer division to get whole number days

    # Save the figure
    filename = f'seeds_day_{day_id}_3D.png'
    plt.savefig(filename, dpi=300)  # Save as high-resolution PNG file

# Create the color bar
# cbar = plt.colorbar(scatter, ax=ax, orientation='vertical')
# cbar.set_label('Temperature')

Plot Cells

In [ ]:
import optimaltransportsolver as ots
from pysdot import PowerDiagram
import pyvista as pv

# Time to plot at
i = 192
# Set path to save the image
image_path = 'TVel_4_A=-0.5.png'  # Specify your desired image file path
# Toggle for showing color bar
show_color_bar = False  # Set this to False to hide, True to show

# Pick colouring
coloring = 'TVel' # Chose how to colour the cells can be temperature ('Temp'), meridional velocity ('MVel') 

# Define a dictionary to map the coloring options to cell data attribute names
coloring_to_attr_name = {
    'Temp': 'Temperature',
    'MVel': 'Meridional Velocity',
    'ZVel': 'Zonal Velocity',
    'TVel': 'Magnitude of Total Velocity'
}

#Construct domain
domain = ots.make_domain(box, False, False, False)

#Draw the tessellation
Lx, Ly, Lz = [abs(box[i+3] - box[i]) for i in range(3)]
pd = PowerDiagram(positions = Z[i], weights = W[i], domain = domain)

# Save the results in a .vtk file
filename = "./data/cells.vtk"
pd.display_vtk(filename)

# Store the colouring intervals
if coloring == 'MVel':
    MVel = [mv[~np.isnan(mv)] for mv in MVel]
    minval = np.min(MVel.flatten())
    maxval = np.max(MVel.flatten())
elif coloring == 'ZVel':
    ZVel = [zv[~np.isnan(zv)] for zv in ZVel]
    minval = np.min(ZVel.flatten())
    maxval = np.max(ZVel.flatten())
elif coloring == 'TVel':
    TVel = [tv[~np.isnan(tv)] for tv in TVel]
    minval = np.min(TVel)
    maxval = np.max(TVel)
elif coloring == 'Temp':
    T = [t[~np.isnan(t)] for t in T]
    minval = np.min(T)
    maxval = np.max(T)
else:
    raise ValueError('Please specify how you want to colour the cells')

# Remove NaN entries from positions and corresponding weights
valid_mask = ~np.isnan(Z[i]).any(axis=1)
Zmod = Z[i][valid_mask]
Wmod = W[i][valid_mask]

# Store the volumes in an array
if coloring == 'MVel':
    colours = np.array(MVel[i])
elif coloring == 'ZVel':
    colours = np.array(ZVel[i])
elif coloring == 'TVel':
    colours = np.array(TVel[i])
elif coloring == 'Temp':
    colours = np.array(T[i]) 
else:
    raise ValueError('Please specify how you want to colour the cells')

# Read the data
grid=pv.read(filename)

# Create cell data that gives the cell volumes, this allows us to colour by cell the velocity or the temperature
cell_colours = colours[grid.cell_data['num'].astype(int)]
grid.cell_data[coloring_to_attr_name[coloring]] = cell_colours

# plot the data with an automatically created plotter, for a static picture use backend='static'
plotter = pv.Plotter(window_size=[800,800], notebook = False, off_screen=True)
plotter.add_mesh(grid, clim = [minval, maxval], cmap = 'jet', show_scalar_bar=show_color_bar)

# Adjust color bar settings if it is shown
if show_color_bar:
    plotter.scalar_bar.SetOrientationToVertical()
    plotter.scalar_bar.SetPosition(0.9, 0.1)  # Position it on the right side

# Hide the grid by turning off the bounding box
plotter.show_bounds(grid='back', location='outer', xtitle='X', ytitle='Y', show_zaxis=False, n_xlabels=3, n_ylabels=3,
                    font_size=30)

# Set the camera for 2D view
plotter.camera_position = 'xy'
plotter.window_size = [4000, 4000]

# Render the frame and save the screenshot
plotter.show()
plotter.screenshot(image_path, transparent_background = True)  # Save the screenshot

Next animate the cells

In [ ]:
import optimaltransportsolver as ots
from pysdot import PowerDiagram
import pyvista as pv
import imageio.v2 as iio

#Animate the cells

coloring = 'TVel' # Chose how to colour the cells can be temperature ('Temp'), meridional velocity ('MVel'), zonal velocity ('ZVel'), or magnitude of total velocity ('TVel')
camera = 'Spin' # Decide whether the camera is looking at the top or bottom of the domain
upper_threshold_percentage = 70 # Send to 100 to plot all 
lower_threshold_percentage = 25 # Send to 0 to plot all

# Define a dictionary to map the coloring options to cell data attribute names
coloring_to_attr_name = {
    'Temp': 'Temperature',
    'MVel': 'Meridional Velocity',
    'ZVel': 'Zonal Velocity',
    'TVel': 'Magnitude of Total Velocity'
}

# Construct the domain
D = ots.make_domain(box, False, False, False)

# Set up the animation parameters
n_frames = len(W)  # Number of frames
angle_increment = 360 / n_frames

# Create an empty list to store frames
frames = []

# Store the colouring intervals
if coloring == 'MVel':
    MVel = [mv[~np.isnan(mv)] for mv in MVel]
    minval = np.min(MVel.flatten())
    maxval = np.max(MVel.flatten())
elif coloring == 'ZVel':
    ZVel = [zv[~np.isnan(zv)] for zv in ZVel]
    minval = np.min(ZVel.flatten())
    maxval = np.max(ZVel.flatten())
elif coloring == 'TVel':
    TVel = [tv[~np.isnan(tv)] for tv in TVel]
    minval = np.min(TVel)
    maxval = np.max(TVel)
elif coloring == 'Temp':
    T = [t[~np.isnan(t)] for t in T]
    minval = np.min(T)
    maxval = np.max(T)
else:
    raise ValueError('Please specify how you want to colour the cells')

# Set the colouring threshold
upper_threshold_value = maxval * (upper_threshold_percentage / 100.0)
lower_threshold_value = maxval * (lower_threshold_percentage / 100.0)

# Generate frames for the animation
for i in range(n_frames):

    # Remove NaN entries from positions and corresponding weights
    valid_mask = ~np.isnan(Z[i]).any(axis=1)
    Zmod = Z[i][valid_mask]
    Wmod = W[i][valid_mask]

    #Draw the tessellation
    pd = PowerDiagram(positions = Zmod , weights = Wmod , domain = D)

    # Save the results in a .vtk file
    filename = "./data/cells.vtk"
    pd.display_vtk(filename)

    # Store the volumes in an array
    if coloring == 'MVel':
        colours = np.array(MVel[i])
    elif coloring == 'ZVel':
        colours = np.array(ZVel[i])
    elif coloring == 'TVel':
        colours = np.array(TVel[i])
    elif coloring == 'Temp':
        colours = np.array(T[i]) 
    else:
        raise ValueError('Please specify how you want to colour the cells')

    # Read the data
    grid=pv.read(filename)

    # Create cell data that gives the cell volumes, this allows us to colour the cells
    opacities = np.where((colours >= lower_threshold_value) & (colours <= upper_threshold_value), 1, 0)  # Fully opaque in the interval, transparent outside it 
    cell_opacities = opacities[grid.cell_data['num'].astype(int)]
    cell_colours = colours[grid.cell_data['num'].astype(int)]
    grid.cell_data[coloring_to_attr_name[coloring]] = cell_colours
    grid.cell_data['opacity'] = cell_opacities

    # plot the data with an automatically created plotter, for a static picture use backend='static'
    plotter = pv.Plotter(window_size=[800,800], notebook = False, off_screen=True)
    plotter.add_mesh(grid, clim = [minval, maxval], cmap = 'jet', opacity = 'opacity')

    # Set the camera for a 2D view at either the bottom or the top
    if camera == 'Angled':
        pass
    elif camera == 'Top':
        plotter.camera_position = 'xy'
    elif camera == 'Bottom':
        plotter.camera_position = 'xy'
        plotter.camera.elevation = 180
        plotter.camera.roll += 360
    elif camera == 'Spin':
        angle = i * angle_increment
        x = 11 * np.sin(np.radians(angle))  # Radius and sine for x-coordinate
        y = 11 * np.cos(np.radians(angle))  # Radius and cosine for y-coordinate
        plotter.camera_position = [(x, y, 11), (0, 0, 0), (0, 0, 1)]
    else:
        raise ValueError('Please specify how you want the camera oriented')

    # Render the frame
    plotter.show()

    # Get the frame as an image array
    frame = plotter.screenshot(transparent_background=True)

    # Add the frame to the list of frames
    frames.append(frame)

# Save the frames as an animation file
output_file = './animations/SG_Cells.gif'
iio.mimwrite(output_file, frames, format = 'gif', fps = 30)

# Combine and syncronize the animations

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from PIL import Image

# Function to read GIF frames
def read_gif(gif_path):
    img = Image.open(gif_path)
    frames = []
    try:
        while True:
            frames.append(img.copy())
            img.seek(len(frames))
    except EOFError:
        pass
    return frames

# Paths to your GIFs
gif1_path = './animations/SG_Cells_TVel.gif'
gif2_path = './animations/SG_Seeds_2D.gif'
#gif3_path = './animations/SG_Centroids_2D.gif'

# Read GIF frames
frames1 = read_gif(gif1_path)
frames2 = read_gif(gif2_path)
#frames3 = read_gif(gif3_path)

# Create a figure and axes
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
# fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for ax in axes:
    ax.axis('off') # Turn off the axis

fig.subplots_adjust(wspace=0.1)  # Adjust the width space

# Display each GIF on a separate subplot
ims = []

for ax, frames in zip(axes, [frames1, frames2]): #, frames3]):
    im = ax.imshow(frames[0], animated=True)
    ims.append([im])

# Update function for the animation
def update(frame):
    for im, frames in zip(ims, [frames1, frames2]): #, frames3]):
        im[0].set_array(frames[frame])

# Create the animation
ani = animation.FuncAnimation(fig, update, frames=len(frames1), repeat=False, blit=False)

# Save the animation (uncomment the line below to save)
ani.save('./animations/Combined_Animation.gif', writer='pillow', fps=30, dpi=175)